In [2]:
from geneticalgs import BinaryGA

import random

# import pandas as pd
import numpy as np
import csv
import time
import pickle

import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
# Make the graphs a bit prettier, and bigger
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [21]:
def generate_diff_weights(n):
    """Generates different random integers from interval (1, 5*n + 1)."""
    interval = (1, 5*n)
    
    random_number = random.randrange(interval[0], interval[1])
    used_values = [random_number]

    for i in range(1, n):
        while random_number in used_values:
            random_number = random.randrange(interval[0], interval[1])

        used_values.append(random_number)

    return used_values

def generate_weights(n):
    """Generates random integers (possible with duplicates) from interval (1, 5*n + 1)."""
    interval = (1, 5*n)
    w = []
    
    for i in range(n):
        w.append(random.randrange(interval[0], interval[1]))
        
    return w

def read_sat(filename):
    with open(filename, 'r') as src_file:
        counter = 0
        clause_list = []

        for line in src_file:
            elems = line.strip().split(" ")

            if elems[0] == 'p':
                num_vars = int(elems[2])
                num_clauses = int(elems[-1])

#                 print('num_vars:', num_vars)
#                 print('num_clauses:', num_clauses)
                continue

            if elems[0] == 'c':
                continue

            if elems[0] == '%':
                break

    #         print(elems)
            clause = []
            # we have 3SAT and so we read three values
            for i in range(3):
                clause.append(int(elems[i]))
#             print(clause)

            clause_list.append(clause)
            counter += 1

        if counter != num_clauses:
            raise ValueError('The given amount of clauses is not equal to the specified one.')
            
        return clause_list, num_vars, num_clauses

In [23]:
path = './data/uf50-218/'
filename = 'uf50-01.cnf'

sat_clauses, num_vars, num_clauses = read_sat(path + filename)
print('num_vars:', num_vars)
print('num_clauses:', num_clauses)

# generate weights
weights = generate_diff_weights(num_vars)

num_vars: 50
num_clauses: 218


In [35]:
def fitness_func(chromosome, data):
    score = 0  # amount of satisfied clauses
    active_vars = [data[idx] for idx in chromosome]  # vars evaluated to 1
    
    for clause in sat_clauses:
        evaluated = False
        
        for elem in clause:
            if not evaluated:
                if elem < 0 and abs(elem) not in active_vars:
                    score += 1
                    evaluated = True
                elif elem > 0 and abs(elem) in active_vars:
                    score += 1
                    evaluated = True
            else:
                break
                
#     print(score)
    sol_weight = 0
    if score == num_clauses:
        for var in active_vars:
            sol_weight += weights[var - 1]
            
    return score + sol_weight

In [36]:
optim = 'max'
elitism = True
input_data = list(range(1, num_vars + 1))

pop_size = 10000
r_sel = 'rank'
t_sel = 'tournament'
# tournament size
# whole population = pop_size * 3 -> this size will be adjusted to population size
t_size = [pop_size * 0.2, pop_size * 0.4, pop_size * 0.6, pop_size * 0.8, pop_size * 3]
mut_prob = 0.05
mut_type = 1
cross_prob = 0.95
cross_type = 1

In [37]:
rank_ga = BinaryGA(input_data, fitness_func, optim=optim, selection=r_sel, 
                   mut_prob=mut_prob, mut_type=mut_type, 
                   cross_prob=cross_prob, cross_type=cross_type, 
                   elitism=elitism)

tour_ga = []
for size in t_size:
    tour_ga.append(BinaryGA(input_data, fitness_func, optim=optim, selection=t_sel, 
                            mut_prob=mut_prob, mut_type=mut_type, 
                            cross_prob=cross_prob, cross_type=cross_type, 
                            elitism=elitism, 
                            tournament_size=size))

In [ ]:
rank_ga.init_random_population(pop_size)

for i in range(len(t_size)):
    tour_ga[i].init_random_population(pop_size)